In [1]:
import pandas as pd

# Load CSV
csv_path = 'full_dataset.csv'
data = pd.read_csv(csv_path)

# Filter for test videos
test_data = data[data['set'] == 'test']

In [2]:
test_data.head()

,path,label,resolution,frame_count,fps,set
6529,YouTube-real/00170.mp4,1,NaN,NaN,NaN,test
6530,YouTube-real/00208.mp4,1,NaN,NaN,NaN,test
6531,YouTube-real/00063.mp4,1,NaN,NaN,NaN,test
6532,YouTube-real/00024.mp4,1,NaN,NaN,NaN,test
6533,YouTube-real/00021.mp4,1,NaN,NaN,NaN,test


In [3]:
import cv2
import numpy as np

def get_frames(video_path, num_frames=100):
    cap = cv2.VideoCapture(video_path)
    frames = []
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

    frame_idxs = np.linspace(0, total_frames - 1, num_frames, dtype=int)

    for frame_idx in frame_idxs:
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_idx)
        ret, frame = cap.read()
        if ret:
            # Resize and normalize frame here if necessary
            frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            frame = cv2.resize(frame, (224, 224))  # Assuming your model uses 224x224 input
            frame = frame / 255.0  # Normalize if your model expects [0,1] range inputs
            frames.append(frame)
    cap.release()

    return np.array(frames)


In [4]:
import torch

def predict_video_label(video_path, model, device):
    frames = get_frames(video_path)
    frames = torch.tensor(frames).permute(0, 3, 1, 2)  # Change (N, H, W, C) to (N, C, H, W)
    frames = frames.float().to(device)  # Ensure the tensor is float and on the correct device

    model.eval()
    with torch.no_grad():
        outputs = model(frames)
        _, predicted_labels = torch.max(outputs, dim=1)
    
    # Majority voting
    votes = predicted_labels.cpu().numpy()
    video_label = np.bincount(votes).argmax()

    return video_label


In [ ]:
def evaluate_model_on_videos(test_data, model, device):
    correct = 0
    total = len(test_data)

    for _, row in test_data.iterrows():
        video_path = row['path']
        true_label = row['label']
        predicted_label = predict_video_label(video_path, model, device)

        if predicted_label == true_label:
            correct += 1

    accuracy = correct / total
    return accuracy



# Load the best model weights
model.load_state_dict(torch.load('best_model.pth'))

# Use the model and device previously defined
test_accuracy = evaluate_model_on_videos(test_data, model, device)
print(f"Test Accuracy: {test_accuracy:.2f}")
